In [1]:
## required imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
## read data
df = pd.read_csv('Data/Twitter_5_year.csv')

In [3]:
# Shape
df.shape 

(299998, 15)

In [4]:
# Head
df.head()

,Account,User Name,Followers at Posting,Created,Type,Likes,Retweets,URL,Message,Screen Name,Link 1,Final Link 1,Link 2,Final Link 2,Score
0,juicy j,therealjuicyj,2645413.0,2019-06-13 23:49:25 EDT,Tweet,864,190,https://twitter.com/therealjuicyj/status/11393...,Whomever made bets made good money tonight,NaN,NaN,NaN,NaN,NaN,-3.59
1,Luke Combs 🎤,lukecombs,336828.0,2019-06-13 23:18:23 EDT,Tweet,3859,102,https://twitter.com/lukecombs/status/113937147...,.@nicohocking is bae,NaN,NaN,NaN,NaN,NaN,1.14
2,ADAM LAMBERT,adamlambert,2477885.0,2019-06-13 23:03:07 EDT,Link,832,343,https://twitter.com/adamlambert/status/1139367...,Korea! Tickets are on sale NOW for Queen + Ada...,NaN,http://ticket.yes24.com!,http://ticket.yes24.com!,https://t.co/CKCDd7BAK5,https://twitter.com/adamlambert/status/1139367...,-1.33
3,Russ,russdiemon,2043124.0,2019-06-13 22:56:16 EDT,Tweet,986,31,https://twitter.com/russdiemon/status/11393659...,What questions y’all want @KidSuper to ask me?...,NaN,NaN,NaN,NaN,NaN,-9.22
4,Gerardo Ortiz,gerardoortiznet,3082321.0,2019-06-13 22:43:58 EDT,Photo,727,76,https://twitter.com/gerardoortiznet/status/113...,🚘😎😜 https://t.co/4yeXAIbwA7,NaN,https://t.co/4yeXAIbwA7,https://twitter.com/gerardoortiznet/status/113...,NaN,NaN,1.89


In [5]:
# Print columns
df.columns

Index(['Account', 'User Name', 'Followers at Posting', 'Created', 'Type',
       'Likes', 'Retweets', 'URL', 'Message', 'Screen Name', 'Link 1',
       'Final Link 1', 'Link 2', 'Final Link 2', 'Score'],
      dtype='object')

In [6]:
df.nunique()

Account                    128
User Name                  128
Followers at Posting     86577
Created                 298457
Type                         6
Likes                    24456
Retweets                 13291
URL                     299998
Message                 295701
Screen Name                142
Link 1                  225630
Final Link 1            223468
Link 2                   32123
Final Link 2             31749
Score                     6753
dtype: int64

In [7]:
df['Type'].value_counts()

Link            146122
Photo            61678
Tweet            56976
Native Video     24707
Video            10486
Vine                29
Name: Type, dtype: int64

In [8]:
df.count()

Account                 299998
User Name               299998
Followers at Posting    290677
Created                 299998
Type                    299998
Likes                   299998
Retweets                299998
URL                     299998
Message                 299998
Screen Name             294460
Link 1                  243040
Final Link 1            243040
Link 2                   39680
Final Link 2             39680
Score                   299998
dtype: int64

In [9]:
sum(df['Followers at Posting'].isna())

9321

In [10]:
df_selected_columns = df[['User Name', 'Followers at Posting', 'Created', 'Type',
       'Likes', 'Retweets','Message']]
df_selected_columns.head()

,User Name,Followers at Posting,Created,Type,Likes,Retweets,Message
0,therealjuicyj,2645413.0,2019-06-13 23:49:25 EDT,Tweet,864,190,Whomever made bets made good money tonight
1,lukecombs,336828.0,2019-06-13 23:18:23 EDT,Tweet,3859,102,.@nicohocking is bae
2,adamlambert,2477885.0,2019-06-13 23:03:07 EDT,Link,832,343,Korea! Tickets are on sale NOW for Queen + Ada...
3,russdiemon,2043124.0,2019-06-13 22:56:16 EDT,Tweet,986,31,What questions y’all want @KidSuper to ask me?...
4,gerardoortiznet,3082321.0,2019-06-13 22:43:58 EDT,Photo,727,76,🚘😎😜 https://t.co/4yeXAIbwA7


In [11]:
df_selected_columns.count()

User Name               299998
Followers at Posting    290677
Created                 299998
Type                    299998
Likes                   299998
Retweets                299998
Message                 299998
dtype: int64

In [12]:
df_selected_columns = df_selected_columns.dropna()
df_selected_columns.reset_index(drop=True, inplace=True)

In [13]:
df_selected_columns.count()

User Name               290677
Followers at Posting    290677
Created                 290677
Type                    290677
Likes                   290677
Retweets                290677
Message                 290677
dtype: int64

In [14]:
# Convert "Created" into datetime 
df_selected_columns['Created'] = pd.to_datetime(df_selected_columns['Created'])

In [15]:
df_selected_columns.head()

,User Name,Followers at Posting,Created,Type,Likes,Retweets,Message
0,therealjuicyj,2645413.0,2019-06-14 03:49:25,Tweet,864,190,Whomever made bets made good money tonight
1,lukecombs,336828.0,2019-06-14 03:18:23,Tweet,3859,102,.@nicohocking is bae
2,adamlambert,2477885.0,2019-06-14 03:03:07,Link,832,343,Korea! Tickets are on sale NOW for Queen + Ada...
3,russdiemon,2043124.0,2019-06-14 02:56:16,Tweet,986,31,What questions y’all want @KidSuper to ask me?...
4,gerardoortiznet,3082321.0,2019-06-14 02:43:58,Photo,727,76,🚘😎😜 https://t.co/4yeXAIbwA7


In [16]:
df_selected_columns['Number of characters'] = df_selected_columns['Message'].apply(len)

In [17]:
df_selected_columns['Number of hashtags'] = df_selected_columns.apply(lambda x: x['Message'].count('#'), axis=1)

In [18]:
df_selected_columns.head()

,User Name,Followers at Posting,Created,Type,Likes,Retweets,Message,Number of characters,Number of hashtags
0,therealjuicyj,2645413.0,2019-06-14 03:49:25,Tweet,864,190,Whomever made bets made good money tonight,42,0
1,lukecombs,336828.0,2019-06-14 03:18:23,Tweet,3859,102,.@nicohocking is bae,20,0
2,adamlambert,2477885.0,2019-06-14 03:03:07,Link,832,343,Korea! Tickets are on sale NOW for Queen + Ada...,232,1
3,russdiemon,2043124.0,2019-06-14 02:56:16,Tweet,986,31,What questions y’all want @KidSuper to ask me?...,81,0
4,gerardoortiznet,3082321.0,2019-06-14 02:43:58,Photo,727,76,🚘😎😜 https://t.co/4yeXAIbwA7,27,0


In [19]:
pd.get_dummies(df_selected_columns, prefix=['Type'], columns=['Type'])

,User Name,Followers at Posting,Created,Likes,Retweets,Message,Number of characters,Number of hashtags,Type_Link,Type_Native Video,Type_Photo,Type_Tweet,Type_Video,Type_Vine
0,therealjuicyj,2645413.0,2019-06-14 03:49:25,864,190,Whomever made bets made good money tonight,42,0,0,0,0,1,0,0
1,lukecombs,336828.0,2019-06-14 03:18:23,3859,102,.@nicohocking is bae,20,0,0,0,0,1,0,0
2,adamlambert,2477885.0,2019-06-14 03:03:07,832,343,Korea! Tickets are on sale NOW for Queen + Ada...,232,1,1,0,0,0,0,0
3,russdiemon,2043124.0,2019-06-14 02:56:16,986,31,What questions y’all want @KidSuper to ask me?...,81,0,0,0,0,1,0,0
4,gerardoortiznet,3082321.0,2019-06-14 02:43:58,727,76,🚘😎😜 https://t.co/4yeXAIbwA7,27,0,0,0,1,0,0,0
5,thalia,9567561.0,2019-06-14 02:19:47,622,140,¡Sorpresa! Surprise!!!! 🎉💃🏼💋 #TRetro #Thalia #...,192,9,0,0,0,0,1,0
6,xtina,16978546.0,2019-06-14 02:16:41,4752,446,Comin for ya Vegas 💋 https://t.co/EB3ecdNRTD,44,0,0,1,0,0,0,0
7,adamlambert,2477885.0,2019-06-14 02:03:36,1370,346,To keep you wanting more my dear. :) it’s like...,214,0,1,0,0,0,0,0
8,adamlambert,2477885.0,2019-06-14 02:01:55,809,234,I actually haven’t decided yet. :) https://twi...,94,0,1,0,0,0,0,0
9,adamlambert,2477885.0,2019-06-14 02:00:57,754,201,No not exactly. https://twitter.com/adaml_dail...,74,0,1,0,0,0,0,0


In [20]:
df_selected_columns['Score'] = df_selected_columns.apply(lambda x: (x['Likes'] + x['Retweets']) / x['Followers at Posting'], axis=1)

In [21]:
df_selected_columns.head()

,User Name,Followers at Posting,Created,Type,Likes,Retweets,Message,Number of characters,Number of hashtags,Score
0,therealjuicyj,2645413.0,2019-06-14 03:49:25,Tweet,864,190,Whomever made bets made good money tonight,42,0,0.000398
1,lukecombs,336828.0,2019-06-14 03:18:23,Tweet,3859,102,.@nicohocking is bae,20,0,0.011760
2,adamlambert,2477885.0,2019-06-14 03:03:07,Link,832,343,Korea! Tickets are on sale NOW for Queen + Ada...,232,1,0.000474
3,russdiemon,2043124.0,2019-06-14 02:56:16,Tweet,986,31,What questions y’all want @KidSuper to ask me?...,81,0,0.000498
4,gerardoortiznet,3082321.0,2019-06-14 02:43:58,Photo,727,76,🚘😎😜 https://t.co/4yeXAIbwA7,27,0,0.000261


In [22]:
df_selected_columns['Score'].describe()

count    290677.000000
mean          0.002099
std           0.008438
min           0.000000
25%           0.000063
50%           0.000245
75%           0.001200
max           1.488100
Name: Score, dtype: float64

In [23]:
df_selected_columns = df_selected_columns.drop(['Followers at Posting', 'Likes', 'Retweets', 'Message'], axis=1)

In [24]:
df_selected_columns.head()

,User Name,Created,Type,Number of characters,Number of hashtags,Score
0,therealjuicyj,2019-06-14 03:49:25,Tweet,42,0,0.000398
1,lukecombs,2019-06-14 03:18:23,Tweet,20,0,0.011760
2,adamlambert,2019-06-14 03:03:07,Link,232,1,0.000474
3,russdiemon,2019-06-14 02:56:16,Tweet,81,0,0.000498
4,gerardoortiznet,2019-06-14 02:43:58,Photo,27,0,0.000261


In [25]:
df_selected_columns.sort_values(by=['User Name','Created'], inplace=True)

In [26]:
df_selected_columns['Time Difference between Prior And Current Post'] = df_selected_columns.groupby('User Name')['Created'].diff()

In [27]:
df_selected_columns.reset_index(drop=True, inplace=True)

In [28]:
df_selected_columns.to_csv('Data/twitter_5yrs_features.csv')